In [2]:
import os
import sys

import numpy as np
import pandas as pd
from copy import deepcopy
from pprint import pprint
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.colors as colors
from hydra import initialize, compose
from glob import glob
from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import build_default_calc, build_default_source

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.helpers import pandeia_input, pyhalo
from mejiro.lenses.test import SampleSkyPyLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.utils import util
from mejiro.analysis import stats

ImportError: cannot import name 'isiterable' from 'astropy.cosmology.utils' (/data/bwedig/.conda/envs/mejiro/lib/python3.10/site-packages/astropy/cosmology/utils.py)

In [ ]:
csv_path = os.path.join('/data','bwedig', 'roman-population', 'data', 'skypy_output.csv')
df = pd.read_csv(csv_path)

In [ ]:
df.column
df.head(10)

,velodisp,massstel,angleins,redssour,redslens,xposlens,yposlens,xpossour,ypossour,numbimag,magnsour,maxmdistimag,magtlensF106,magtsourF106,magtsourMagnifiedF106
0,251.025397,0.036141,1.084403,2.310156,0.616690,-0.141015,0.043382,-1.106556,0.981286,1.0,1.369664,NaN,21.885183,21.923449,21.581914
1,251.025397,0.036141,0.617908,1.048704,0.616690,-0.063282,0.000795,0.290395,1.652574,1.0,1.480503,NaN,21.885183,23.000643,22.574621
2,251.025397,0.036141,0.558357,0.981603,0.616690,0.085376,-0.123559,0.574170,1.027743,1.0,1.840321,NaN,21.885183,23.436977,22.774743
3,275.859863,0.189665,0.575309,1.277251,0.840802,-0.122405,-0.099575,0.229196,-0.471078,2.0,2.217466,1.233549,21.062455,23.321226,22.456583
4,275.859863,0.189665,1.010629,2.136921,0.840802,-0.014995,0.037117,-0.674593,0.819483,2.0,2.172031,2.122993,21.062455,23.948160,23.105994
5,275.859863,0.189665,0.498959,1.194140,0.840802,-0.023646,0.017482,1.256379,1.366837,1.0,1.358797,NaN,21.062455,23.570487,23.237601
6,275.859863,0.189665,0.951266,1.955572,0.840802,-0.063334,0.173066,-0.855267,-0.712657,1.0,1.522384,NaN,21.062455,23.193224,22.736914
7,275.859863,0.189665,0.706240,1.451394,0.840802,-0.014367,0.029347,1.423275,0.001612,1.0,1.319173,NaN,21.062455,23.611526,23.310772
8,258.235759,0.076367,1.146890,3.127759,0.737675,-0.118176,0.111632,-0.082367,-0.716826,2.0,1.761327,2.209537,21.592352,23.556204,22.941604
9,258.235759,0.076367,1.033841,2.353918,0.737675,-0.129202,0.013501,-0.004000,0.996112,2.0,1.876932,1.999783,21.592352,23.343351,22.659729
